In [ ]:
from models.DistMult import DistMult_Lite
from models.Complex import Complex
from models.ConvE import ConvE, ConvE_args

from utils.loaders import load_data, get_onehots
from utils.evaluation_metrics import SRR, auprc_auroc_ap

import torch
import numpy as np
from sklearn.utils import shuffle

In [ ]:
from utils.path_manage import get_files

data, lookup, ASD_dictionary, BCE_dictionary, _, __ = get_files()
entities = int(len(lookup)/2)

In [ ]:
len(set(data[:,2]))

In [ ]:
preprocessed_target = torch.LongTensor(subjects[index])

In [ ]:
torch.squeeze(torch.stack([(protien_ids == x).nonzero() for x in preprocessed_target]),1)

In [ ]:
protien_ids = list(set(data[:,2]))
protien_ids = torch.LongTensor(protien_ids)

In [ ]:
number_of_batches = 5
number_of_epochs = 20
x = shuffle(data)
test_data = x[:50]

In [ ]:
model = DistMult_Lite(num_entities = entities, embedding_dim=100, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(number_of_epochs):


        #training stage 
        model.train()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            # print(obj.shape)
            rel = torch.LongTensor(relationships[index])
            # print(len(rel))
            
            preprocessed_target = torch.squeeze(torch.LongTensor(subjects[index]))
            target = torch.squeeze(torch.stack([(protien_ids == x).nonzero() for x in preprocessed_target]))
            # print(len(subj))


            optimiser.zero_grad()
            pred = model.forward(obj = obj, rel = rel, subj = protien_ids)
            
            loss = model.loss(pred, target)
            
            loss.backward()
            optimiser.step()


        #evaluation stage
        model.eval()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)
        total_sum_reciporical_rank = torch.zeros(1)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            
            preprocessed_target = torch.squeeze(torch.LongTensor(subjects[index]))
            target = torch.squeeze(torch.stack([(protien_ids == x).nonzero() for x in preprocessed_target]),1)

            predictions = model.forward(obj = obj, rel = rel, subj = protien_ids)
            srr = SRR(predictions, target) 
            # print(predictions.shape)
            # print(targets.shape)
            total_sum_reciporical_rank = total_sum_reciporical_rank + srr

        print('mean reciporical rank is...')
        print(total_sum_reciporical_rank / len(test_data))
